# 第 10 章 自然语言处理入门

##10.1 分词

### 10.1.1 英文分词

In [18]:
import tensorflow.keras.preprocessing.text as kp_text

paragraph = "The 5 biggest countries by population in 2017 are China, " \
            "India, United States, Indonesia, and Brazil."
processed_text = kp_text.text_to_word_sequence(paragraph)
print(processed_text)

# 输出如下：
# ['the', '5', 'biggest', 'countries', 'by', 'population', 'in', '2017',
#  'are', 'china', 'india', 'united', 'states', 'indonesia', 'and', 'brazil']

['the', '5', 'biggest', 'countries', 'by', 'population', 'in', '2017', 'are', 'china', 'india', 'united', 'states', 'indonesia', 'and', 'brazil']


### 10.1.2 中文分词

In [19]:
import jieba

seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所")  # 搜索引擎模式
print(", ".join(seg_list))

我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
我/ 来到/ 北京/ 清华大学
他, 来到, 了, 网易, 杭研, 大厦
小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所


## 10.2 语言模型

### 10.2.1 独热编码

In [20]:
token2idx = {'人工智能': 0, '的': 1, '研究': 2, '可以': 3, '分为': 4, '几个': 5,
             '技术': 6, '问题': 7, '是': 8, '一门': 9, '新': 10, '科学': 11}

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

samples = ["人工智能 的 研究 可以 分为 几个 技术 问题", "人工智能 是 一门 新 的 技术 科学"]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(samples)

print(tokenizer.word_index)

sequence = tokenizer.texts_to_sequences(samples)
print(sequence)

print(to_categorical(sequence[0],
                     num_classes=len(tokenizer.word_index)+1))

{'人工智能': 1, '的': 2, '技术': 3, '研究': 4, '可以': 5, '分为': 6, '几个': 7, '问题': 8, '是': 9, '一门': 10, '新': 11, '科学': 12}
[[1, 2, 4, 5, 6, 7, 3, 8], [1, 9, 10, 11, 2, 3, 12]]
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


### 10.2.2 词嵌入

In [22]:
import gensim

model_path = 'data/word2vec/sgns.weibo.bigram-char'
w2v = gensim.models.KeyedVectors.load_word2vec_format(model_path)
vector = w2v['猫咪']
print(vector)

PermissionError: [Errno 13] Permission denied: 'data/word2vec/sgns.weibo.bigram-char'

In [ ]:
# 输出词向量词表前 20 个词语
print(f"word list: {w2v.index2word[:20]}")

# 输出词向量前 20 个词的向量，形状为 (20, 300)
print(f"word vectors: {w2v.vectors[:20]}")

vector = w2v['猫咪']
print(f"most similiar to 猫咪: \n {w2v.similar_by_vector(vector)}")

print(f"most similiar to 明星: \n {w2v.most_similar('明星')}")

In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(input_dim=1000,  # 标记个数，这个嵌入层总共能嵌入 999 个标记
                            output_dim=128)  # 嵌入维度

### 10.2.3 从文本到词嵌入

In [ ]:
import gensim
import numpy as np
from typing import List
from tensorflow import keras

model_path = 'data/word2vec/sgns.weibo.bigram-char'
# 通常预训练词嵌入会比较大，加载很耗时耗内存资源，当内存资源有限或者需要快速实验时
# 可以通过增加一个 limit 参数，只读取特定数量词向量来节省时间和资源
# 下面代码只会加载最高频的 5000 个词的向量
w2v = gensim.models.KeyedVectors.load_word2vec_format(model_path, limit=5000)

token2index = {
    '<PAD>': 0, # 由于我们用 0 补全序列，所以补全标记的索引必须为 0
    '<UNK>': 1  # 新词标记的索引可以使任何一个，设置为 1 只是为了方便
}

# 我们遍历预训练词嵌入的词表，加入到我们的标记索引词典
for token in w2v.index2word:
    token2index[token] = len(token2index)

# 初始化一个形状为 [标记总数，预训练向量维度] 的全 0 张量
token_vector = np.zeros((len(token2index), w2v.vector_size))
# 随机初始化 <UNK> 标记的张量
token_vector[1] = np.random.rand(300)
# 从索引 2 开始使用预训练的向量
token_vector[2:] = w2v.vectors

# 通过测试可以确定新构建的标记索引和标记向量映射关系没问题
print(token_vector[token2index['成长']] == w2v['成长'])
print(token_vector[token2index['市场']] == w2v['市场'])

In [ ]:
# 使用处理过的预训练向量来初始化嵌入层
L = keras.layers
embedding_layer = L.Embedding(input_dim=len(token2index),  # 标记数量等于词表标记数量
                              output_dim=w2v.vector_size,  # 嵌入维度等于预训练向量维度
                              weights=[token_vector],        # 使用我们构建的权重张量
                              trainable=False)             # 不可训练
# 构建一个提取序列向量的模型
model = keras.Sequential([
    embedding_layer
])
# 我们不需要训练这个模型，所以这里的损失函数和优化器可以随意设定
model.compile('adam', 'sparse_categorical_crossentropy')
model.summary()

In [ ]:
def convert_token_2_idx(tokenized_sentence: List[str]) -> List[int]:
    """转换分词后的标记序列为标记索引序列

    如果该标记在词表出现过使用其索引，如果词表不存在，则使用新词标记的索引来替代
    Args:
        tokenized_sentence: 分词后的序列
    Returns:
        标记索引序列
    """
    token_ids = []
    for token in tokenized_sentence:
        token_ids.append(token2index.get(token, token2index['<UNK>']))
    return token_ids

tokenized_sentence = "今天 天气 真 不错 ha".split(' ')
print(convert_token_2_idx(tokenized_sentence))

In [ ]:
sentence_index = convert_token_2_idx(tokenized_sentence)
# 将序列索引包含一个样本的批量
input_x = np.array([sentence_index])
# 使用模型预测
sentence_vector = model.predict(input_x)
print(sentence_vector.shape)

In [ ]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))